Test llama_index

In [ ]:
import os
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index import StorageContext, load_index_from_storage
import sklearn


os.environ["OPENAI_API_KEY"] = "XXXXX"

In [ ]:
import re

lines_seen = set() # holds lines already seen
lines_per_chunk = 1000 # number of lines per chunk
chunk_counter = 1

outfile = open('vero_no_duplicates_{}.txt'.format(chunk_counter), "w")
line_counter = 0

for line in open('og_data/vero_text.txt', "r"):
    # not a duplicate and does not match unwanted patterns
    if line not in lines_seen and not re.match(r'^[\d,]*$', line): 
        outfile.write(line)
        lines_seen.add(line)
        line_counter += 1
        if line_counter >= lines_per_chunk:
            outfile.close()
            chunk_counter += 1
            outfile = open('data/vero_no_duplicates_{}.txt'.format(chunk_counter), "w")
            line_counter = 0

outfile.close()

In [ ]:
documents = SimpleDirectoryReader('devdata').load_data()

In [ ]:
documents

In [ ]:
index = GPTVectorStoreIndex.from_documents(documents)

In [ ]:
index.set_index_id("vero_index")
index.storage_context.persist('./storage')

In [ ]:
query_engine = index.as_query_engine()

# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine(
        vector_store_query_mode="svm"
    )
response = query_engine.query(
        "Mikä on henkilötietodirektiivin 95/46/EY ?"
    )


In [ ]:
str(response)

Test langchain
tools with llama index

In [ ]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

In [ ]:


tools = [
    Tool(
        name = "GPT Index",
        func=lambda q: str(query_engine.query(q).get_formatted_sources()),
        description="useful for when you want to answer questions about the author. The input to this tool should be a complete english sentence.",
        return_direct=True
    ),
]



In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0)
agent_executor = initialize_agent(tools, llm, agent="conversational-react-description", memory=memory)

In [ ]:
agent_executor.run(input="Mikä on henkilötietodirektiivin 95/46/EY ?")